In [1]:
import torch
import sys


import os
os.environ['HSA_OVERRIDE_GFX_VERSION'] = '10.3.0' # Adjust to your GPU's GFX version

print(f"Python 版本: {sys.version}")
print(f"PyTorch 版本: {torch.__version__}")
print("-" * 30)

if torch.cuda.is_available():
    print("GPU (CUDA/ROCm) 可用！")
    print(f"GPU 数量: {torch.cuda.device_count()}")
    for i in range(torch.cuda.device_count()):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
        try:
            print(f"  计算能力 (Compute Capability): {torch.cuda.get_device_capability(i)}")
        except Exception as cap_e:
            print(f"  无法获取计算能力: {cap_e}")
        try:
            print(f"  总显存: {torch.cuda.get_device_properties(i).total_memory / (1024**3):.2f} GB")
        except Exception as mem_e:
             print(f"  无法获取显存信息: {mem_e}")
else:
    print("GPU (CUDA/ROCm) 不可用。请检查：")
    print("1. 是否安装了支持 GPU (CUDA 或 ROCm) 的 PyTorch 版本？")
    print("   (对于 AMD GPU, 需要 ROCm 版本的 PyTorch)")
    print("2. GPU 驱动是否正确安装？")
    print("   (对于 AMD GPU, 需要 AMD ROCm 驱动)")
    print("3. ROCm 版本是否与 PyTorch 版本兼容？")
    print("4. 你的 AMD GPU 型号是否被 ROCm 和 PyTorch 支持？")

print("-" * 30)
try:
    if torch.cuda.is_available():
        tensor = torch.tensor([1.0, 2.0]).cuda()
        print("成功将张量移动到 GPU:", tensor)
        print("当前设备:", tensor.device)
    else:
        print("无法将张量移动到 GPU，因为 GPU (CUDA/ROCm) 不可用。")
except Exception as e:
    print(f"尝试将张量移动到 GPU 时出错: {e}")
    

Python 版本: 3.11.12 (main, Apr  9 2025, 04:04:00) [Clang 20.1.0 ]
PyTorch 版本: 2.7.1+rocm6.3
------------------------------
GPU (CUDA/ROCm) 可用！
GPU 数量: 1
GPU 0: AMD Radeon RX 6750 GRE 12GB
  计算能力 (Compute Capability): (10, 3)
  总显存: 11.98 GB
------------------------------
成功将张量移动到 GPU: tensor([1., 2.], device='cuda:0')
当前设备: cuda:0


In [2]:
import torch, grp, pwd, os, subprocess

import os
os.environ['HSA_OVERRIDE_GFX_VERSION'] = '10.3.0' # Adjust to your GPU's GFX version

devices = []
try:
	print("\n\nChecking ROCM support...")
	result = subprocess.run(['rocminfo'], stdout=subprocess.PIPE)
	cmd_str = result.stdout.decode('utf-8')
	cmd_split = cmd_str.split('Agent ')
	for part in cmd_split:
		item_single = part[0:1]
		item_double = part[0:2]
		if item_single.isnumeric() or item_double.isnumeric():
			new_split = cmd_str.split('Agent '+item_double)
			device = new_split[1].split('Marketing Name:')[0].replace('  Name:                    ', '').replace('\n','').replace('                  ','').split('Uuid:')[0].split('*******')[1]
			devices.append(device)
	if len(devices) > 0:
		print('GOOD: ROCM devices found: ', len(devices))
	else:
		print('BAD: No ROCM devices found.')

	print("Checking PyTorch...")
	x = torch.rand(5, 3)
	has_torch = False
	len_x = len(x)
	if len_x == 5:
		has_torch = True
		for i in x:
			if len(i) == 3:
				has_torch = True
			else:
				has_torch = False
	if has_torch:
		print('GOOD: PyTorch is working fine.')
	else:
		print('BAD: PyTorch is NOT working.')


	print("Checking user groups...")
	user = os.getlogin()
	groups = [g.gr_name for g in grp.getgrall() if user in g.gr_mem]
	gid = pwd.getpwnam(user).pw_gid
	groups.append(grp.getgrgid(gid).gr_name)
	if 'render' in groups and 'video' in groups:
		print('GOOD: The user', user, 'is in RENDER and VIDEO groups.')
	else:
		print('BAD: The user', user, 'is NOT in RENDER and VIDEO groups. This is necessary in order to PyTorch use HIP resources')

	if torch.cuda.is_available():
		print("GOOD: PyTorch ROCM support found.")
		t = torch.tensor([5, 5, 5], dtype=torch.int64, device='cuda')
		print('Testing PyTorch ROCM support...')
		if str(t) == "tensor([5, 5, 5], device='cuda:0')":
			print('Everything fine! You can run PyTorch code inside of: ')
			for device in devices:
				print('---> ', device)
	else:
		print("BAD: PyTorch ROCM support NOT found.")
except:
	print('Cannot find rocminfo command information. Unable to determine if AMDGPU drivers with ROCM support were installed.')



Checking ROCM support...
GOOD: ROCM devices found:  2
Checking PyTorch...
GOOD: PyTorch is working fine.
Checking user groups...
GOOD: The user jagger is in RENDER and VIDEO groups.
GOOD: PyTorch ROCM support found.
Testing PyTorch ROCM support...
Everything fine! You can run PyTorch code inside of: 
--->  AMD Ryzen 5 7500F 6-Core Processor   
--->  gfx1030            
